In [65]:
import os
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df = pd.read_csv(r"D:\01.Educación\03.MMA\03.Fall 2024\02.Decision analytics (MGSC 662)\05.Project\02.Project solar panels distribution\finaldataset.csv")
df['PercentagePark'] = df['PercentagePark'].fillna(0)
df['GridAll'] = df['GridAll'].fillna(0)

**Changes**

- Budget to 4 billion
- Districts not connect to grid produce 100% of its demand
- Districts cannot produce over 200% of its demand
- Only grid connected districts can import

# **Optimization Model for Solar Panel Installation**

## **Decision Variables**
- $X_i$: Installed capacity in district $i$ (MW).
- $Z_i$: Total number of panels installed in district $i$ (integer).
- $T_{k,i}$: Number of panels transported from supplier $k$ to district $i$ (integer).
- $y_i$: Binary variable indicating whether panels are installed in district $i$ ($1$ if installed, $0$ otherwise).
- $A_{j,i}$: Energy transferred from district $j$ to district $i$ (MW).
- $E_i$: Total energy produced in district $i$ (MW).
- $I_i$: Imported electricity in district $i$ (MW).
- $s_i$: Surplus energy in district $i$ (MW).
- $E_{\text{exp},i}$: Exportable energy from district $i$ (MW).
- $\delta_i$: Binary variable indicating if district $i$ exports energy.

---

## **Parameters**
- $EP$: Energy production per panel (MW).
- $S_i$: Solar potential for district $i$ (MW/MWp/month).
- $ED_i$: Energy demand in district $i$ (MW/month).
- $D_{k,i}$: Distance from supplier $k$ to district $i$ (km).
- $D_{j,i}$: Distance from district $j$ to district $i$ (km).
- $C_{\text{install}}$: Installation cost per MW ($).
- $C_T$: Transportation cost per panel per km ($).
- $B$: Total budget ($).
- $M$: Large constant for binary constraints.
- $\gamma$: Transmission loss factor per km.
- $C_{\text{max}}$: Maximum transmission capacity (MW/month).
- $\alpha_i$: Demand satisfaction ratio for district $i$:
$$
\alpha_i = 
\begin{cases} 
0.05, & \text{if district $i$ is grid-connected} \\
1, & \text{otherwise}
\end{cases}
$$
- $GridAll_i$: Indicator if district $i$ is grid-connected ($1$ if connected, $0$ otherwise).

---

## **Objective Function**

The objective is to minimize the total cost of solar panel installation and transportation:

$$
\min Z = \sum_{i=1}^{n} X_i \cdot C_{\text{install}} + \sum_{k=1}^{m} \sum_{i=1}^{n} T_{k,i} \cdot C_T \cdot D_{k,i}
$$

### **Explanation of Variables and Parameters:**
- $Z$: Total cost of installation and transportation.
- $X_i$: Installed capacity in district $i$ (MW).
- $C_{\text{install}}$: Installation cost per MW ($).
- $T_{k,i}$: Number of panels transported from supplier $k$ to district $i$.
- $C_T$: Transportation cost per panel per kilometer ($).
- $D_{k,i}$: Distance from supplier $k$ to district $i$ (km).
- $n$: Total number of districts.
- $m$: Total number of suppliers.


---

## **Constraints**

### **1. Panels Balance**
The total number of panels transported to each district equals the number of panels installed:

$$
\sum_{k=1}^{m} T_{k,i} = Z_i, \quad \forall i \in \{1, \dots, n\}
$$

- $T_{k,i}$: Panels transported from supplier $k$ to district $i$.
- $Z_i$: Total number of panels installed in district $i$.

---

### **2. Installed Capacity**
Installed capacity is proportional to the number of panels installed:

$$
X_i = Z_i \cdot EP, \quad \forall i \in \{1, \dots, n\}
$$

- $X_i$: Installed capacity in district $i$ (MW).
- $Z_i$: Total number of panels installed in district $i$.
- $EP$: Energy production per panel (MW).

---

### **3. Demand Satisfaction**
Local production plus imported electricity must meet the adjusted demand:

$$
X_i \cdot S_i + I_i \geq ED_i \cdot \alpha_i, \quad \forall i \in \{1, \dots, n\}
$$

- $S_i$: Solar efficiency factor for district $i$.
- $I_i$: Imported electricity in district $i$ (MW).
- $ED_i$: Energy demand in district $i$ (MW).
- $\alpha_i$: Demand satisfaction ratio for district $i$.

---

### **4. Surplus and Exportable Energy**
Surplus energy in a district is defined as:

$$
s_i = X_i \cdot S_i - ED_i \cdot \alpha_i, \quad \forall i \in \{1, \dots, n\}
$$

Exportable energy is:

$$
E_{\text{exp},i} = s_i \cdot \delta_i, \quad \forall i \in \{1, \dots, n\}
$$

- $s_i$: Surplus energy in district $i$ (MW).
- $E_{\text{exp},i}$: Exportable energy from district $i$ (MW).
- $\delta_i$: Binary variable indicating if district $i$ exports energy.

---

### **5. Imported Electricity**
Imported electricity considers transmission losses:

$$
I_i = \sum_{\substack{j=1 \\ j \neq i}}^{n} A_{j,i} \cdot e^{-\gamma D_{j,i}}, \quad \forall i \in \{1, \dots, n\}
$$

- $A_{j,i}$: Energy transferred from district $j$ to district $i$ (MW).
- $\gamma$: Transmission loss factor per km.
- $D_{j,i}$: Distance from district $j$ to district $i$ (km).

---

### **6. Binary Installation Constraint**
Installed capacity is limited by the installation decision:

$$
X_i \leq M \cdot y_i, \quad \forall i \in \{1, \dots, n\}
$$

- $M$: Large constant for binary constraints.
- $y_i$: Binary variable indicating whether panels are installed in district $i$.

---

### **7. Energy Production for Non-Grid Districts**
For districts without grid connection ($GridAll_i = 0$):

$$
E_i \geq ED_i, \quad \forall i \text{ where } GridAll_i = 0
$$

- $E_i$: Total energy produced in district $i$ (MW).

---

### **8. Maximum Production Limit**
Production in each district cannot exceed twice its demand:

$$
E_i \leq 2 \cdot ED_i, \quad \forall i \in \{1, \dots, n\}
$$

---

### **9. Export Limit by Surplus**
Energy exported cannot exceed exportable energy:

$$
\sum_{\substack{i=1 \\ i \neq j}}^{n} A_{j,i} \leq E_{\text{exp},j}, \quad \forall j \in \{1, \dots, n\}
$$

---

### **10. Transmission Capacity Limit**
Exports are limited by the transmission capacity:

$$
\sum_{\substack{i=1 \\ i \neq j}}^{n} A_{j,i} \leq C_{\text{max}} \cdot \delta_j, \quad \forall j \in \{1, \dots, n\}
$$

- $C_{\text{max}}$: Maximum transmission capacity (MW).

---

### **11. No Self-Transfer**
Districts cannot transfer energy to themselves:

$$
A_{j,j} = 0, \quad \forall j \in \{1, \dots, n\}
$$

---

### **12. Energy Balance**
The total imported energy equals total transmitted energy after losses:

$$
\sum_{i=1}^{n} I_i = \sum_{j=1}^{n} \sum_{\substack{i=1 \\ i \neq j}}^{n} A_{j,i} \cdot e^{-\gamma D_{j,i}}
$$

---

### **13. Budget Constraint**
Total costs must not exceed the budget:

$$
\sum_{i=1}^{n} X_i \cdot C_{\text{install}} + \sum_{k=1}^{m} \sum_{i=1}^{n} T_{k,i} \cdot C_T \cdot D_{k,i} \leq B
$$

- $B$: Total budget ($).

In [66]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

district_ids = df['ID'].tolist()
district_names = df['Name'].tolist()
num_districts = len(df)
supplier_indices = df[df['Suppliers'] == 1].index.tolist()
num_suppliers = len(supplier_indices)

energy_consumption_per_person = (96.3 / 4.5)  # kWh/month divided by average people per household
C_install = 1.1e6  # Installation cost per MW ($)
C_T = 0.1  # Transportation cost per panel per km ($)
B = 4e9  # Budget ($) - updated to 3 billion
transmission_loss_per_km = 0.0377 / 1000  # Loss per km (fraction)
max_transmission_capacity = 500  # MW/month
M = 1e6  # Big M for binary constraints
EP = 0.3 / 1e3  # Panel capacity (MWp)

ED = df['2021_Total_Population'] * energy_consumption_per_person / 1e6  # MW/month
S = df['MEANSOLAR'] * 30 / 1e3  # Convert to MW per MWp per month
P = df['PercentagePark'].values / 100

X_coords = df['X'].values
Y_coords = df['Y'].values
D = np.zeros((num_districts, num_districts))

def haversine(lon1, lat1, lon2, lat2):
    # Calculate distance between two points on Earth
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # Radius of Earth in km
    return km

for i in range(num_districts):
    for j in range(num_districts):
        if i != j:
            D[i, j] = haversine(X_coords[i], Y_coords[j], X_coords[j], Y_coords[i])

# Initialize model
model = gp.Model("Solar_Panels_Optimization")
model.Params.LogToConsole = 0

# Decision Variables
X = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=0, name="InstalledCapacity")
Z = model.addVars(num_districts, vtype=GRB.INTEGER, lb=0, name="TotalPanels")
T = model.addVars(num_suppliers, num_districts, vtype=GRB.INTEGER, lb=0, name="TransportedPanels")
y = model.addVars(num_districts, vtype=GRB.BINARY, name="InstallDecision")
A = model.addVars(num_districts, num_districts, vtype=GRB.CONTINUOUS, lb=0, name="TransferredEnergy")
exportable_energy = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=0, name="ExportableEnergy")
surplus = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, name="Surplus")
delta = model.addVars(num_districts, vtype=GRB.BINARY, name="ExportDecision")
imported_electricity = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=0, name="ImportedElectricity")
E = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=0, name="EnergyProduced")

# Objective: Minimize installation and transportation costs
model.setObjective(
    gp.quicksum(X[i] * C_install for i in range(num_districts)) +
    gp.quicksum(T[k, i] * C_T * D[supplier_indices[k], i] for k in range(num_suppliers) for i in range(num_districts)),
    GRB.MINIMIZE
)

# Define demand satisfaction ratio based on GridConnections
demand_satisfaction_ratio = [
    0.05 if df.loc[i, 'GridAll'] > 0 else 1
    for i in range(num_districts)
]

# Constraints
for i in range(num_districts):
    # Panels balance
    model.addConstr(gp.quicksum(T[k, i] for k in range(num_suppliers)) == Z[i], f"PanelsBalance_District_{i}")
    
    # Installed capacity
    model.addConstr(X[i] == Z[i] * EP, f"InstalledCapacity_District_{i}")

    # Demand satisfaction with adjusted ratio
    local_production = X[i] * S[i]
    model.addConstr(
        local_production + imported_electricity[i] >= ED[i] * demand_satisfaction_ratio[i],
        f"DemandSatisfaction_District_{i}"
    )
    
    # Surplus and exportable energy
    model.addConstr(surplus[i] == local_production - ED[i] * demand_satisfaction_ratio[i], f"Surplus_District_{i}")
    model.addConstr(exportable_energy[i] == surplus[i] * delta[i], f"ExportableEnergy_District_{i}")
    
    # Import electricity
    model.addConstr(
        imported_electricity[i] == gp.quicksum(A[j, i] * np.exp(-transmission_loss_per_km * D[j, i]) for j in range(num_districts) if j != i),
        f"ImportedElectricity_District_{i}"
    )
    
    # Binary installation constraints
    model.addConstr(X[i] <= M * y[i], f"BinaryInstallationConstraint_District_{i}")
    
    # Grid constraint for delta[i]
    if df.loc[i, 'GridAll'] == 0:
        model.addConstr(delta[i] == 0, f"GridConstraint_District_{i}")
    
    # Surplus and Export Decision
    model.addConstr(
        surplus[i] >= 0.0001 - M * (1 - delta[i]),
        f"PositiveSurplusForExport_District_{i}"
    )

    # Energy production constraint for districts without grid (100% of demand)
    if df.loc[i, 'GridAll'] == 0:
        model.addConstr(E[i] >= ED[i], f"MinProductionNoGrid_District_{i}")
    
    # Link energy produced (E[i]) to local production
    model.addConstr(E[i] == X[i] * S[i], f"EnergyProduced_District_{i}")
    
    # Maximum production limit (200% of demand)
    model.addConstr(E[i] <= 2 * ED[i], f"MaxProductionLimit_District_{i}")

for j in range(num_districts):
    # Export Limit by Surplus
    model.addConstr(
        gp.quicksum(A[j, i] for i in range(num_districts) if i != j) <= exportable_energy[j],
        f"ExportLimit_District_{j}"
    )
    # Transmission Capacity Limit
    model.addConstr(
        gp.quicksum(A[j, i] for i in range(num_districts) if i != j) <= max_transmission_capacity * delta[j],
        f"TransmissionCapacity_District_{j}"
    )
    # No Self-Transfer
    model.addConstr(A[j, j] == 0, f"NoSelfTransfer_District_{j}")

# Energy balance
model.addConstr(
    gp.quicksum(imported_electricity[i] for i in range(num_districts)) == 
    gp.quicksum(A[j, i] * np.exp(-transmission_loss_per_km * D[j, i]) for j in range(num_districts) for i in range(num_districts) if i != j),
    "EnergyBalance"
)

# Budget constraint
total_cost = gp.quicksum(X[i] * C_install for i in range(num_districts)) + \
             gp.quicksum(T[k, i] * C_T * D[supplier_indices[k], i] for k in range(num_suppliers) for i in range(num_districts))
model.addConstr(total_cost <= B, "BudgetConstraint")

# Optimize and display results
model.optimize()

if model.status == GRB.OPTIMAL:
    # Total summary
    total_panels_installed = sum(Z[i].x for i in range(num_districts))
    total_installed_capacity = sum(X[i].x for i in range(num_districts))
    total_cost = model.objVal

    print("Optimization Results")
    print("====================")
    print(f"Total Cost: ${total_cost:,.2f}")
    print(f"Total Number of Panels Installed: {total_panels_installed:,.0f}")
    print(f"Total Installed Capacity: {total_installed_capacity:.2f} MW")
    print("\nDetails by District")
    print("====================")
    for i in range(num_districts):
        panels_installed = Z[i].x
        installed_capacity = X[i].x
        local_prod = installed_capacity * S[i]
        imp_elec = imported_electricity[i].x
        demand = ED[i]

        print(f"District {district_names[i]}:")
        print(f"  Panels Installed: {panels_installed:,.0f}")
        print(f"  Installed Capacity: {installed_capacity:.2f} MW")
        print(f"  Local Production: {local_prod:.2f} MW")
        print(f"  Capacity Imported: {imp_elec:.2f} MW")
        print(f"  Demand: {demand:.2f} MW")
        print("-" * 30)
else:
    print("No feasible solution found.")

Optimization Results
Total Cost: $3,300,218,535.28
Total Number of Panels Installed: 9,779,652
Total Installed Capacity: 2933.90 MW

Details by District
District Abim:
  Panels Installed: -0
  Installed Capacity: 0.00 MW
  Local Production: 0.00 MW
  Capacity Imported: 0.20 MW
  Demand: 4.01 MW
------------------------------
District Adjumani:
  Panels Installed: -0
  Installed Capacity: 0.00 MW
  Local Production: 0.00 MW
  Capacity Imported: 0.26 MW
  Demand: 5.26 MW
------------------------------
District Agago:
  Panels Installed: -0
  Installed Capacity: 0.00 MW
  Local Production: 0.00 MW
  Capacity Imported: 0.29 MW
  Demand: 5.76 MW
------------------------------
District Amuru:
  Panels Installed: -0
  Installed Capacity: 0.00 MW
  Local Production: 0.00 MW
  Capacity Imported: 0.25 MW
  Demand: 5.10 MW
------------------------------
District Arua:
  Panels Installed: -0
  Installed Capacity: 0.00 MW
  Local Production: 0.00 MW
  Capacity Imported: 0.18 MW
  Demand: 3.69 MW
--

In [67]:
district_results = {
    "District": district_names,  # Names of districts
    "Panels Installed": [round(Z[i].x, 3) for i in range(num_districts)],  # Total panels installed
    "Installed Capacity (MW)": [round(X[i].x, 3) for i in range(num_districts)],  # Installed capacity
    "Local Production (MW)": [round(X[i].x * S[i], 3) for i in range(num_districts)],  # Local production
    "Capacity Imported (MW)": [round(imported_electricity[i].x, 3) for i in range(num_districts)],  # Electricity imported
    "Demand (MW)": [round(ED[i], 3) for i in range(num_districts)],  # Energy demand
    "Local Production / Demand": [round((X[i].x * S[i]) / ED[i], 3) for i in range(num_districts)]  # Ratio
}

results_df_base = pd.DataFrame(district_results)

# results_df[results_df["Installed Capacity (MW)"] < results_df["Local Production (MW)"]]


In [68]:
results_df_base

,District,Panels Installed,Installed Capacity (MW),Local Production (MW),Capacity Imported (MW),Demand (MW),Local Production / Demand
0,Abim,-0.0,0.000,0.000,0.200,4.009,0.000
1,Adjumani,-0.0,0.000,0.000,0.263,5.263,0.000
2,Agago,-0.0,0.000,0.000,0.288,5.757,0.000
3,Amuru,-0.0,0.000,0.000,0.255,5.099,0.000
4,Arua,-0.0,0.000,0.000,0.185,3.693,0.000
5,Gulu,-0.0,0.000,0.000,0.139,2.785,0.000
6,Koboko,-0.0,0.000,0.000,0.315,6.305,0.000
7,Kole,-0.0,0.000,0.000,0.339,6.772,0.000
8,Kotido,-0.0,0.000,0.000,0.241,4.818,0.000
9,Lira,-0.0,0.000,0.000,0.278,5.564,0.000


In [69]:
#results_df.to_csv("final_allocation.csv")

In [70]:
#Sensitivity function
def test_optimization(
    df, 
    baseline_results,
    budget=None,
    installation_cost=None,
    transportation_cost=None,
    demand_change=None
):
    """
    Test the optimization model under different parameter levels.

    Parameters:
    - df: DataFrame containing district data.
    - baseline_results: Dictionary with baseline optimization results.
    - budget: Modified budget ($).
    - installation_cost: Modified installation cost per MW ($).
    - transportation_cost: Modified transportation cost per panel per km ($).
    - demand_change: Percentual change in demand (e.g., 0.1 for +10%).

    Returns:
    - results_df: DataFrame with optimization results.
    - differences_df: DataFrame with differences from the baseline.
    """

    # Adjust demand if demand_change is provided
    S = df['MEANSOLAR'] * 30 / 1e3  # Solar efficiency remains constant
    X_coords = df['X'].values
    Y_coords = df['Y'].values
    district_ids = df['ID'].tolist()
    district_names = df['Name'].tolist()
    num_districts = len(df)
    supplier_indices = df[df['Suppliers'] == 1].index.tolist()
    num_suppliers = len(supplier_indices)
    energy_consumption_per_person = (96.3 / 4.5)  # kWh/month divided by average people per household
    transmission_loss_per_km = 0.0377 / 1000  # Loss per km (fraction)
    max_transmission_capacity = 500  # MW/month
    M = 1e6  # Big M for binary constraints
    EP = 0.3 / 1e3  # Panel capacity (MWp)
    S = df['MEANSOLAR'] * 30 / 1e3  # Convert to MW per MWp per month
    P = df['PercentagePark'].values / 100
    D = np.zeros((num_districts, num_districts))

    # Modified parameters
    B = budget if budget is not None else 4e9
    C_install = installation_cost if installation_cost is not None else 1.1e6
    C_T = transportation_cost if transportation_cost is not None else 0.1
    ED = (df['2021_Total_Population']* energy_consumption_per_person / 1e6)* (1 + (demand_change if demand_change else 0))  # MW/month

    
    def haversine(lon1, lat1, lon2, lat2):
        # Calculate distance between two points on Earth
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
        c = 2 * np.arcsin(np.sqrt(a))
        km = 6371 * c  # Radius of Earth in km
        return km

    for i in range(num_districts):
        for j in range(num_districts):
            if i != j:
                D[i, j] = haversine(X_coords[i], Y_coords[j], X_coords[j], Y_coords[i])

    # Initialize model
    model = gp.Model("Solar_Panels_Optimization")
    model.Params.LogToConsole = 0

    # Decision Variables
    X = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=0, name="InstalledCapacity")
    Z = model.addVars(num_districts, vtype=GRB.INTEGER, lb=0, name="TotalPanels")
    T = model.addVars(num_suppliers, num_districts, vtype=GRB.INTEGER, lb=0, name="TransportedPanels")
    y = model.addVars(num_districts, vtype=GRB.BINARY, name="InstallDecision")
    A = model.addVars(num_districts, num_districts, vtype=GRB.CONTINUOUS, lb=0, name="TransferredEnergy")
    exportable_energy = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=0, name="ExportableEnergy")
    surplus = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=-GRB.INFINITY, name="Surplus")
    delta = model.addVars(num_districts, vtype=GRB.BINARY, name="ExportDecision")
    imported_electricity = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=0, name="ImportedElectricity")
    E = model.addVars(num_districts, vtype=GRB.CONTINUOUS, lb=0, name="EnergyProduced")

    # Objective: Minimize installation and transportation costs
    model.setObjective(
        gp.quicksum(X[i] * C_install for i in range(num_districts)) +
        gp.quicksum(T[k, i] * C_T * D[supplier_indices[k], i] for k in range(num_suppliers) for i in range(num_districts)),
        GRB.MINIMIZE
    )

    # Define demand satisfaction ratio based on GridConnections
    demand_satisfaction_ratio = [
        0.05 if df.loc[i, 'GridAll'] > 0 else 1
        for i in range(num_districts)
    ]

    # Constraints
    for i in range(num_districts):
        # Panels balance
        model.addConstr(gp.quicksum(T[k, i] for k in range(num_suppliers)) == Z[i], f"PanelsBalance_District_{i}")
        
        # Installed capacity
        model.addConstr(X[i] == Z[i] * EP, f"InstalledCapacity_District_{i}")

        # Demand satisfaction with adjusted ratio
        local_production = X[i] * S[i]
        model.addConstr(
            local_production + imported_electricity[i] >= ED[i] * demand_satisfaction_ratio[i],
            f"DemandSatisfaction_District_{i}"
        )
        
        # Surplus and exportable energy
        model.addConstr(surplus[i] == local_production - ED[i] * demand_satisfaction_ratio[i], f"Surplus_District_{i}")
        model.addConstr(exportable_energy[i] == surplus[i] * delta[i], f"ExportableEnergy_District_{i}")
        
        # Import electricity
        model.addConstr(
            imported_electricity[i] == gp.quicksum(A[j, i] * np.exp(-transmission_loss_per_km * D[j, i]) for j in range(num_districts) if j != i),
            f"ImportedElectricity_District_{i}"
        )
        
        # Binary installation constraints
        model.addConstr(X[i] <= M * y[i], f"BinaryInstallationConstraint_District_{i}")
        
        # Grid constraint for delta[i]
        if df.loc[i, 'GridAll'] == 0:
            model.addConstr(delta[i] == 0, f"GridConstraint_District_{i}")
        
        # Surplus and Export Decision
        model.addConstr(
            surplus[i] >= 0.0001 - M * (1 - delta[i]),
            f"PositiveSurplusForExport_District_{i}"
        )

        # Energy production constraint for districts without grid (100% of demand)
        if df.loc[i, 'GridAll'] == 0:
            model.addConstr(E[i] >= ED[i], f"MinProductionNoGrid_District_{i}")
        
        # Link energy produced (E[i]) to local production
        model.addConstr(E[i] == X[i] * S[i], f"EnergyProduced_District_{i}")
        
        # Maximum production limit (200% of demand)
        model.addConstr(E[i] <= 2 * ED[i], f"MaxProductionLimit_District_{i}")

    for j in range(num_districts):
        # Export Limit by Surplus
        model.addConstr(
            gp.quicksum(A[j, i] for i in range(num_districts) if i != j) <= exportable_energy[j],
            f"ExportLimit_District_{j}"
        )
        # Transmission Capacity Limit
        model.addConstr(
            gp.quicksum(A[j, i] for i in range(num_districts) if i != j) <= max_transmission_capacity * delta[j],
            f"TransmissionCapacity_District_{j}"
        )
        # No Self-Transfer
        model.addConstr(A[j, j] == 0, f"NoSelfTransfer_District_{j}")

    # Energy balance
    model.addConstr(
        gp.quicksum(imported_electricity[i] for i in range(num_districts)) == 
        gp.quicksum(A[j, i] * np.exp(-transmission_loss_per_km * D[j, i]) for j in range(num_districts) for i in range(num_districts) if i != j),
        "EnergyBalance"
    )

    # Budget constraint
    total_cost = gp.quicksum(X[i] * C_install for i in range(num_districts)) + \
                gp.quicksum(T[k, i] * C_T * D[supplier_indices[k], i] for k in range(num_suppliers) for i in range(num_districts))
    model.addConstr(total_cost <= B, "BudgetConstraint")

    # Optimize and display results
    model.optimize()
        
        
        # Collect results if the solution is feasible
    if model.status == GRB.OPTIMAL:
       results = {
             "District": df['Name'].tolist(),
           "Panels Installed": [round(model.getVarByName(f"TotalPanels[{i}]").X, 3) for i in range(num_districts)],
            "Installed Capacity (MW)": [round(model.getVarByName(f"InstalledCapacity[{i}]").X, 3) for i in range(num_districts)],
            "Total Cost": round(model.objVal, 3)
       }
           
           # Calculate differences from baseline
       differences = {
           "District": results["District"],
               "Panels Installed Diff": [
                   results["Panels Installed"][i] - baseline_results["Panels Installed"][i] 
                   for i in range(num_districts)
               ],
               "Installed Capacity Diff (MW)": [
                   results["Installed Capacity (MW)"][i] - baseline_results["Installed Capacity (MW)"][i]
                   for i in range(num_districts)
               ]
           }
            
       results_df = pd.DataFrame(results)
       differences_df = pd.DataFrame(differences)
       # Filter rows where there is at least one positive value
       differences_df = differences_df[(differences_df["Panels Installed Diff"] > 0) | (differences_df["Installed Capacity Diff (MW)"] > 0)]

            
       return results_df, differences_df
    else:
        print("No feasible solution found.")
        return None, None
        

In [72]:
  
results_df,differences = test_optimization(
    df=df,
    baseline_results=results_df_base,
    demand_change=0.1
)

# Display filtered results
print(differences)

          District  Panels Installed Diff  Installed Capacity Diff (MW)
22       Arua City                 2818.0                         0.845
28           Busia                43033.0                        12.909
35          Tororo                30887.0                         9.267
83        Alebtong                14808.0                         4.443
84        Amolatar                 9286.0                         2.786
85          Amudat                 7778.0                         2.333
86            Apac                13223.0                         3.966
87          Dokolo                12050.0                         3.615
88         Kaabong                 7022.0                         2.107
89          Kitgum                12578.0                         3.774
90          Kwania                12183.0                         3.655
91           Lamwo                 8025.0                         2.408
92       Nabilatuk                 5304.0                       